In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
from src.features.rankings import RankingFeatures
from src.utils import load_data_template

In [3]:
data = load_data_template(season=True)
data.dropna(subset=['a_win'], inplace=True)
rank_feat = RankingFeatures(default_lags=1)
data = rank_feat.per_team_wrapper(data, rank_feat.pca_variables_rankings, per_game=False, per_day=True)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(77615, 87)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,PC1_team_a,PC2_team_a,PC3_team_a,...,PC11_team_b_lag-1,PC12_team_b_lag-1,PC13_team_b_lag-1,PC14_team_b_lag-1,PC15_team_b_lag-1,PC16_team_b_lag-1,PC17_team_b_lag-1,PC18_team_b_lag-1,PC19_team_b_lag-1,PC20_team_b_lag-1
148691,2015,1276,1326,False,1,True,111,274.93248,-144.872737,-71.745162,...,-3.532979,-14.913722,4.378104,-18.264476,-1.145399,-21.530255,7.338276,-5.534678,21.234965,-11.276811


In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

Metric Mean: 0.57 (0.00)
